In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
with open(r'\\wsl.localhost\Ubuntu\home\ochensati\cafemol\example\vibrate\vibrate.movie') as f:
    attoms = f.read()
    
segs = attoms.split('ENDMDL')    

steps =[]
for seg in segs:
    proteins = seg.split('<< protein_  2') 
    if len(proteins) == 1:
        continue
    lines = proteins[1].split('\n')
    coords =[]
    for line in lines:
        if line.startswith('ATOM'):
            coords.append(np.array( [float(x) for x in line.split()[6:9]]))
    steps.append(coords)
    
len(steps)
    

0

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
redoxEnergies = {'TRP': .85,
                 'TYR': 1.08,
                # 'PHE': 1.15,  # ? cannot find a value for this, but it seems to be related to redox potentials in metalloproteins
                 'CYS': 1.2,
                 #'HIS': 1.31,
                 #'MET': 1.43,
                 #'BTN': 1.03
                 } 


models = 0
atomDists={}
atoms = {}
positions = ""
firstPositions =""
with open(r'\\wsl.localhost\Ubuntu\home\ochensati\cafemol\example\vibrate\vibrate.movie') as f:
    while True:
        line = f.readline()
        if line=='':
            break
        if line.startswith("MODEL"):
            models +=1
            if models>1:                        
                for atom in atoms:
                    atoms[atom]["coordsCOM"] = []
                    if atom not in  atomDists :
                        atomDists[atom] = []

                chains = list(set ( [x["chain"] for x in atoms.values()]))
                for chain in chains :
                    chainAtoms = [x for x in atoms  if atoms[x]["chain"]==chain]   
                    chainCenterOfMass = np.mean([atoms[ x]["coords"] for x in chainAtoms], axis=0)
                    for atom in chainAtoms:
                        atomDists[atom] .append (    np.linalg.norm( np.array(atoms[ atom]["coords"]) - chainCenterOfMass))
            if firstPositions == "":
                firstPositions = positions
            positions = ""
            atoms = {}
            
        elif line.startswith("ATOM"):
            positions += line +"\n"
            amino = line[17:21].strip()
            #get the residue number
            if amino in redoxEnergies:
                parts = [x for x in line.split(" ") if x!=""]
                index = int(parts[1])
                coord = [float(line[30:38]), float(line[38:46]), float(line[46:54])]
                chain = parts[4]
                if index not in atoms:
                    atoms[index] = { "amino": amino, "chain":chain,   "coords":[]}
                atoms[index]["coords"].append(coord)
                
with open(r'D:\PythonProj\KineticMontiCarlo\2bam.std','w') as f:
    for atom in atomDists:
        std=(np.std(np.array(atomDists[atom]) ))
        print(std*std)
        f.write(f"{atom}\t{std}\n")                
    

In [6]:
pdb= "SA_C4B"
#open tab delimited file and read into a list
with open(r'\\wsl.localhost\Ubuntu\home\ochensati\cafemol\utility\respac_batch\results' + "\\" + pdb +'.charge') as f:
    lines = f.readlines()

attoms=[]    
for i in range(len(lines)):
    attoms.append( lines[i].split(' ')    )

lines=""
for cc in range(len(attoms)):
    lines += f'CHARGE_CHANGE    {attoms[cc][0]}    {attoms[cc][1].strip()}\n'    
    
#write list to tab delimited file
#with open(r'D:\PythonProj\KineticMontiCarlo' + "\\" + pdb +'.charge', 'w') as f:
#        f.write( lines)
print(lines)        

CHARGE_CHANGE    1    1.53837
CHARGE_CHANGE    2    -0.986614
CHARGE_CHANGE    3    0.630013
CHARGE_CHANGE    4    -0.896463
CHARGE_CHANGE    5    1.39053
CHARGE_CHANGE    6    -0.858001
CHARGE_CHANGE    7    -0.130787
CHARGE_CHANGE    8    0.682411
CHARGE_CHANGE    9    -0.905559
CHARGE_CHANGE    10    0.795476
CHARGE_CHANGE    11    -0.525091
CHARGE_CHANGE    12    0.872216
CHARGE_CHANGE    14    -0.146697
CHARGE_CHANGE    15    0.0189434
CHARGE_CHANGE    17    -0.147394
CHARGE_CHANGE    18    0.115285
CHARGE_CHANGE    19    0.26063
CHARGE_CHANGE    20    0.650965
CHARGE_CHANGE    21    -0.520931
CHARGE_CHANGE    23    0.268022
CHARGE_CHANGE    24    -0.103267
CHARGE_CHANGE    25    -0.270215
CHARGE_CHANGE    28    -0.320979
CHARGE_CHANGE    32    -0.46428
CHARGE_CHANGE    35    -0.344811
CHARGE_CHANGE    38    -0.0950619
CHARGE_CHANGE    39    0.195253
CHARGE_CHANGE    40    0.467132
CHARGE_CHANGE    41    -0.36672
CHARGE_CHANGE    42    0.0646467
CHARGE_CHANGE    43    -0.504334
CH

In [8]:
nProt = 5 

for i in range(1,nProt+1):
    print(f"NLOCAL({i}/{i}) AICG2_PLUS EXV")

for i in range(1,nProt+1):
    for j in range (i+1,nProt+1):
        print(f'NLOCAL({i}/{j}) EXV ELE HP')
       

NLOCAL(1/1) AICG2_PLUS EXV
NLOCAL(2/2) AICG2_PLUS EXV
NLOCAL(3/3) AICG2_PLUS EXV
NLOCAL(4/4) AICG2_PLUS EXV
NLOCAL(5/5) AICG2_PLUS EXV
NLOCAL(1/2) EXV ELE HP
NLOCAL(1/3) EXV ELE HP
NLOCAL(1/4) EXV ELE HP
NLOCAL(1/5) EXV ELE HP
NLOCAL(2/3) EXV ELE HP
NLOCAL(2/4) EXV ELE HP
NLOCAL(2/5) EXV ELE HP
NLOCAL(3/4) EXV ELE HP
NLOCAL(3/5) EXV ELE HP
NLOCAL(4/5) EXV ELE HP


In [ ]:
hp =[]
elec= []
repul=[]
with open(r'\\wsl.localhost\Ubuntu\home\ochensati\cafemol\example\vibrate\vibrate.ts','r') as f:
    while True:
        parts = f.readline()
        if not parts:
            break 
        parts=parts .strip().split(' ')
        cols =[]
        for part in parts:#remove empty strings
            if part != '':
                cols.append(part)
        if parts[0]=='#all':
            hp.append( float(cols[-1]))
            elec.append( float(cols[-2]))
            repul.append( float(cols[-3]))
        
plt.plot(elec,label='Elec')
plt.plot(hp,label='HP')   
plt.plot(repul,label='Repul')
plt.legend()
plt.show()      